In [3]:
from aalpy.utils import generate_random_deterministic_automata
from aalpy.SULs import AutomatonSUL
from aalpy.oracles import RandomWMethodEqOracle, RandomWalkEqOracle
from aalpy.learning_algs import run_KV, run_Lstar

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import SimpleRNN

2025-02-04 18:22:54.866862: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Exact learning of the Reber DFA used in 
Cleeremans, Axel, David Servan-Schreiber, and James L. McClelland. "Finite state automata and simple recurrent networks." Neural computation 1.3 (1989): 372-381.

In [6]:
# The Reber dfa from Reber, A.S. 1967. Implicit learning of artificial grammars. I. Verbal Leartiing Verbal Behau. 5, 855-863.
# A sink state is added to make it a proper dfa
def get_reber_dfa():
    from aalpy.automata import Dfa
    
    reber_dfa = {
        'q0': (False, {'t': 'q1', 'p': 'q2', 's': 'sink', 'x': 'sink', 'v': 'sink'}),
        'q1': (False, {'s': 'q1', 'x': 'q3', 't': 'sink', 'p': 'sink', 'v': 'sink'}),
        'q2': (False, {'t': 'q2', 'v': 'q4', 's': 'sink', 'x': 'sink', 'p': 'sink'}),
        'q3': (False, {'s': 'q5', 'x': 'q2', 't': 'sink', 'p': 'sink', 'v': 'sink'}),
        'q4': (False, {'p': 'q3', 'v': 'q5', 's': 'sink', 'x': 'sink', 't': 'sink'}),
        'q5': (True, {'t': 'sink', 'p': 'sink', 's': 'sink', 'x': 'sink', 'v': 'sink'}),
        'sink': (False, {'t': 'sink', 'p': 'sink', 's': 'sink', 'x': 'sink', 'v': 'sink'})
    }

    return Dfa.from_state_setup(reber_dfa)


In [9]:
# Using RandomWalkEqOracle
reber_dfa= get_reber_dfa()
alphabet = reber_dfa.get_input_alphabet()

sul = AutomatonSUL(reber_dfa)
eq_oracle = RandomWalkEqOracle(alphabet, sul, 1500)

learned_dfa = run_Lstar(alphabet, sul, eq_oracle, automaton_type='dfa',
                        cache_and_non_det_check=True, cex_processing=None, print_level=3)

assert learned_dfa == reber_dfa

print(reber_dfa)

print(learned_dfa)

Hypothesis 1: 1 states.
------------------------
Prefixes / E set |()    
------------------------
()               |False 
------------------------
('t',)           |False 
------------------------
('p',)           |False 
------------------------
('s',)           |False 
------------------------
('x',)           |False 
------------------------
('v',)           |False 
------------------------
Counterexample ('p', 'v', 'v')
Hypothesis 2: 4 states.
----------------------------------------------------
Prefixes / E set     |()    |('v',) |('v', 'v')     
----------------------------------------------------
()                   |False |False  |(False, False) 
----------------------------------------------------
('p',)               |False |False  |(False, True)  
----------------------------------------------------
('p', 'v')           |False |True   |(True, False)  
----------------------------------------------------
('p', 'v', 'v')      |True  |False  |(False, False) 
----------------

In [10]:
# Using RandomWMethodEqOracle
reber_dfa= get_reber_dfa()
alphabet = reber_dfa.get_input_alphabet()

sul = AutomatonSUL(reber_dfa)
eq_oracle = RandomWMethodEqOracle(alphabet, sul, 1500)

learned_dfa = run_Lstar(alphabet, sul, eq_oracle, automaton_type='dfa',
                        cache_and_non_det_check=True, cex_processing=None, print_level=3)

assert learned_dfa == reber_dfa

print(reber_dfa)

print(learned_dfa)

Hypothesis 1: 1 states.
------------------------
Prefixes / E set |()    
------------------------
()               |False 
------------------------
('t',)           |False 
------------------------
('p',)           |False 
------------------------
('s',)           |False 
------------------------
('x',)           |False 
------------------------
('v',)           |False 
------------------------
Counterexample ('t', 'x', 's')
Hypothesis 2: 4 states.
----------------------------------------------------
Prefixes / E set     |()    |('s',) |('x', 's')     
----------------------------------------------------
()                   |False |False  |(False, False) 
----------------------------------------------------
('t',)               |False |False  |(False, True)  
----------------------------------------------------
('t', 'x')           |False |True   |(False, False) 
----------------------------------------------------
('t', 'x', 's')      |True  |False  |(False, False) 
----------------

-----------------------------------
Learning Finished.
Learning Rounds:  4
Number of states: 7
Time (in seconds)
  Total                : 0.48
  Learning algorithm   : 0.02
  Conformance checking : 0.46
Learning Algorithm
 # Membership Queries  : 440
 # MQ Saved by Caching : 139
 # Steps               : 3830
Equivalence Query
 # Membership Queries  : 10500
 # Steps               : 105522
-----------------------------------
digraph learnedModel {
q0 [label="q0"];
q1 [label="q1"];
q2 [label="q2"];
q3 [label="q3"];
q4 [label="q4"];
q5 [label="q5", shape=doublecircle];
sink [label="sink"];
q0 -> q1 [label="t"];
q0 -> q2 [label="p"];
q0 -> sink [label="s"];
q0 -> sink [label="x"];
q0 -> sink [label="v"];
q1 -> q1 [label="s"];
q1 -> q3 [label="x"];
q1 -> sink [label="t"];
q1 -> sink [label="p"];
q1 -> sink [label="v"];
q2 -> q2 [label="t"];
q2 -> q4 [label="v"];
q2 -> sink [label="s"];
q2 -> sink [label="x"];
q2 -> sink [label="p"];
q3 -> q5 [label="s"];
q3 -> q2 [label="x"];
q3 -> sink [lab

## RNN learning of the Reber dfa following the methodology of Cleeremans et al.

In [ ]:
# Cleeremans et al. seem to be using a one-hot encoding of the five letters p,s,t,v,x. 
# Will use [0,0,0,0,0] for none of them
dim_in = 5; dim_out = 5

# Set up training data because of the sink state we've added we can use any random element of {p,s,t,v,x}*
x_train = 


model=Sequential()
# They mention 3 hidden units (sec 2, p3)
model.add(SimpleRNN(input_shape=(None, dim_in), 
                    return_sequences=True, 
                    units=3))
model.add(TimeDistributed(Dense(activation='sigmoid',
                                units=dim_out)))



In [4]:
# Function returning random strings
from scipy.stats import poisson
import numpy as np

def get_random_strings(rate, size, chars):
    random_strings = []
    # sample the lengths of the random strings
    lengths = poisson.rvs(mu = rate, size=size)
    # get the right amount of characters
    symbols = np.random.choice(chars, np.sum(lengths))
    k = 0
    for l in lengths:
        random_strings.append(''.join(symbols[k:k+l-1]))
        k+=l
    return random_strings

# Function testing membership
def query(automaton, word):
    # Empty string for DFA
    if len(word) == 0:
        out = [automaton.step(None)]
    else:
        out = [automaton.step(letter) for letter in word]
    return out

In [8]:
reber_dfa= get_reber_dfa()
output = query(reber_dfa, 'txst')
print(output)

[False, False, True, False]
